# Test update of CMIP6 data: MIROC

**Rooki** calls climate data operations on the **rook** processing service.

In [1]:
import os
import time
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import operators as ops

## Load intake catalog for cmip6

In [2]:
import intake

In [3]:
cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat = intake.open_catalog(cat_url)
df_cmip6 = cat['c3s-cmip6'].read()
df_cmip6.head()

/usr/local/Caskroom/mambaforge/base/envs/rooki/lib/python3.11/site-packages/dask/dataframe/io/csv.py:542: UserWarning: Warning gzip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


,ds_id,path,size,mip_era,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,start_time,end_time,bbox,level
0,c3s-cmip6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp245....,ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/A...,28037112,c3s-cmip6,ScenarioMIP,MOHC,UKESM1-0-LL,ssp245,r1i1p1f2,Amon,ts,gn,v20190507,2015-01-16T00:00:00,2049-12-16T00:00:00,"0.94, -89.38, 359.06, 89.38",NaN
1,c3s-cmip6.ScenarioMIP.MOHC.UKESM1-0-LL.ssp245....,ScenarioMIP/MOHC/UKESM1-0-LL/ssp245/r1i1p1f2/A...,38838222,c3s-cmip6,ScenarioMIP,MOHC,UKESM1-0-LL,ssp245,r1i1p1f2,Amon,ts,gn,v20190507,2050-01-16T00:00:00,2100-12-16T00:00:00,"0.94, -89.38, 359.06, 89.38",NaN
2,c3s-cmip6.ScenarioMIP.NCAR.CESM2.ssp370.r4i1p1...,ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/pr...,104081588,c3s-cmip6,ScenarioMIP,NCAR,CESM2,ssp370,r4i1p1f1,Amon,pr,gn,v20200528,2015-01-15T12:00:00,2064-12-15T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN
3,c3s-cmip6.ScenarioMIP.NCAR.CESM2.ssp370.r4i1p1...,ScenarioMIP/NCAR/CESM2/ssp370/r4i1p1f1/Amon/pr...,74977662,c3s-cmip6,ScenarioMIP,NCAR,CESM2,ssp370,r4i1p1f1,Amon,pr,gn,v20200528,2065-01-15T12:00:00,2100-12-15T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN
4,c3s-cmip6.ScenarioMIP.AS-RCEC.TaiESM1.ssp370.r...,ScenarioMIP/AS-RCEC/TaiESM1/ssp370/r1i1p1f1/Am...,144277888,c3s-cmip6,ScenarioMIP,AS-RCEC,TaiESM1,ssp370,r1i1p1f1,Amon,rlut,gn,v20201014,2015-01-16T12:00:00,2100-12-16T12:00:00,"0.00, -90.00, 358.75, 90.00",NaN


In [4]:
df = df_cmip6.loc[
    (df_cmip6.experiment_id=="historical")
    & (df_cmip6.variable_id=="tas") 
    & (df_cmip6.table_id=="Amon")
    & (df_cmip6.institution_id=="MIROC")
    & (df_cmip6.source_id=="MIROC-ES2H")
]
dsets = list(df.ds_id.unique())
dsets

['c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p1f2.Amon.tas.gn.v20210125',
 'c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.tas.gn.v20220322']

## Run subset

https://docs.xarray.dev/en/stable/generated/xarray.concat.html

In [5]:
start = time.time()



wf = ops.Subset(
        ops.Input('ds', ['c3s-cmip6.CMIP.MIROC.MIROC-ES2H.historical.r1i1p4f2.Amon.tas.gn.v20220322']),
        time="2000/2000"
)

resp = wf.orchestrate()

end = time.time()
print(f"duration: {int(end - start)} secs")
print(resp)

resp.ok

duration: 6 secs
Metalink URL: http://rook4.cloud.dkrz.de:80/outputs/rook/2375386c-10e0-11ee-9037-fa163ed6c06f/input.meta4, num files: 1


True

**open with xarray**

In [6]:
resp.size_in_mb

0.8748559951782227

In [7]:
resp.download_urls()

['http://rook4.cloud.dkrz.de:80/outputs/rook/2575dd7e-10e0-11ee-8112-fa163ed6c06f/tas_Amon_MIROC-ES2H_historical_r1i1p4f2_gn_20000116-20001216.nc']

In [8]:
dsets = resp.datasets()

In [9]:
ds = dsets[0]
ds

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, lat: 128, lon: 256)
Coordinates:
  * time       (time) datetime64[ns] 2000-01-16T12:00:00 ... 2000-12-16T12:00:00
  * lat        (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon        (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/44)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2022-03-08T06:23:10Z
    ...                     ...
    title:                  MIROC-ES2H output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p4f2
    license:                CMIP6 model data produced by MIROC is licensed un...
    cmor_version:           3.3.2
    tracking_id:            hdl:21.14100/38bb043f-41f6-43f4-b987-5a4af051fe14

**provenance**

In [10]:
prov_plot_url = resp.provenance_image()
prov_plot_url

'http://rook4.cloud.dkrz.de:80/outputs/rook/2375386c-10e0-11ee-9037-fa163ed6c06f/provenance.png'

In [11]:
# from IPython.display import Image
# Image(prov_plot_url)

In [12]:
resp.provenance()

'http://rook4.cloud.dkrz.de:80/outputs/rook/2375386c-10e0-11ee-9037-fa163ed6c06f/provenance.json'